In [10]:
import pandas as pd
import numpy as np

In [163]:
query = pd.read_pickle('get_player_data.pkl')
prepared_query = query.format(season="2018", partition_date="20181025")
df = pd.read_gbq(prepared_query, project_id='scarlet-labs', dialect='standard')

C:\Users\neste\Anaconda3\lib\site-packages\oauth2client\_helpers.py:255: UserWarning: Cannot access bigquery_credentials.dat: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/v2/auth?client_id=495642085510-k0tmvj2m941jhre2nbqka17vqpjfddtd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&access_type=offline&response_type=code

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.
Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 24629 rows.

Total time taken 6.7 s.
Finished at 2018-10-31 23:02:47.


In [150]:
most_recent_df = df.groupby('bbrefID', as_index=False, group_keys=False).max()[['bbrefID', 'date']].reset_index().drop(['index'], axis=1)

In [168]:
def get_rolling_game_avgs(df1, index_on, games=20):
    _df = df1.groupby(index_on, as_index=False, group_keys=False).rolling(games).mean().reset_index().drop(["index"], axis=1).fillna(0)
    df_transformed = _df.set_index(["date"] + index_on).select_dtypes([np.number])
    new_col_names = ["{col}_{index_type}_{rolling}g".format(col=c,
                                                            index_type = '_'.join(index_on),
                                                            rolling=games) 
                     for c in df_transformed.columns]
    df_transformed.columns = new_col_names
    return df_transformed

In [169]:
player_feature_list = [get_rolling_game_avgs(df, index_on=['bbrefID'], games=g) for g in [5,10,20]]
opp_feature_list = [get_rolling_game_avgs(df, index_on=['opp', 'pos'], games=g) for g in [5,10,20]]

In [158]:
player_feature_df = pd.concat(player_feature_list, axis=1)
opp_feature_df = pd.concat(opp_feature_list, axis=1).reset_index().groupby(['date', 'opp', 'pos']).median()

In [159]:
tmp = df.set_index(['date', 'bbrefID']).join(player_feature_df).reset_index().set_index(['date', 'opp', 'pos']).join(opp_feature_df).reset_index()

In [161]:
tmp

,date,opp,pos,bbrefID,player,tm,G,GS,venue,secs_played,...,DRB_pct_opp_pos_20g,TRB_pct_opp_pos_20g,AST_pct_opp_pos_20g,STL_pct_opp_pos_20g,BLK_pct_opp_pos_20g,TOV_pct_opp_pos_20g,USG_pct_opp_pos_20g,ORtg_opp_pos_20g,DRtg_opp_pos_20g,GmSc_opp_pos_20g
0,2017-10-17,BOS,C,loveke01,Kevin Love,CLE,1,1,home,1704,...,17.0425,12.8250,5.9100,1.1400,1.5950,0.0000,14.6225,103.175,104.900,5.8675
1,2017-10-17,BOS,C,thomptr01,Tristan Thompson,CLE,1,0,home,1176,...,17.0425,12.8250,5.9100,1.1400,1.5950,0.0000,14.6225,103.175,104.900,5.8675
2,2017-10-17,BOS,PF,greenje02,Jeff Green,CLE,1,0,home,854,...,21.5750,13.4275,10.4125,1.3375,2.1175,12.4175,19.1925,99.925,105.175,8.3400
3,2017-10-17,BOS,PF,jamesle01,LeBron James,CLE,1,1,home,2472,...,21.5750,13.4275,10.4125,1.3375,2.1175,12.4175,19.1925,99.925,105.175,8.3400
4,2017-10-17,BOS,SG,korveky01,Kyle Korver,CLE,1,0,home,439,...,13.3275,8.7200,11.2775,1.5450,0.3800,0.0000,19.0075,83.750,106.975,5.8775
5,2017-10-17,BOS,SG,shumpim01,Iman Shumpert,CLE,1,0,home,771,...,13.3275,8.7200,11.2775,1.5450,0.3800,0.0000,19.0075,83.750,106.975,5.8775
6,2017-10-17,BOS,SG,smithjr01,J.R. Smith,CLE,1,0,home,1315,...,13.3275,8.7200,11.2775,1.5450,0.3800,0.0000,19.0075,83.750,106.975,5.8775
7,2017-10-17,BOS,SG,wadedw01,Dwyane Wade,CLE,1,1,home,1710,...,13.3275,8.7200,11.2775,1.5450,0.3800,0.0000,19.0075,83.750,106.975,5.8775
8,2017-10-17,CLE,C,baynear01,Aron Baynes,BOS,1,0,away,1146,...,18.4425,14.4525,13.5750,2.2725,3.2525,0.0000,16.1400,130.225,113.550,7.5550
9,2017-10-17,CLE,C,horfoal01,Al Horford,BOS,1,1,away,1927,...,18.4425,14.4525,13.5750,2.2725,3.2525,0.0000,16.1400,130.225,113.550,7.5550


In [171]:
most_recent_df.set_index(['bbrefID', 'date']).join(tmp.set_index(['bbrefID', 'date'])).reset_index()

,bbrefID,date,opp,pos,player,tm,G,GS,venue,secs_played,...,DRB_pct_opp_pos_20g,TRB_pct_opp_pos_20g,AST_pct_opp_pos_20g,STL_pct_opp_pos_20g,BLK_pct_opp_pos_20g,TOV_pct_opp_pos_20g,USG_pct_opp_pos_20g,ORtg_opp_pos_20g,DRtg_opp_pos_20g,GmSc_opp_pos_20g
0,abrinal01,2018-04-09,MIA,SG,Alex Abrines,OKC,75,0,away,706,...,11.0475,6.4700,10.0025,2.1825,0.8200,8.6725,20.3350,108.300,104.400,8.2575
1,acyqu01,2018-04-11,BOS,PF,Quincy Acy,BRK,70,0,away,1414,...,21.5150,13.3650,9.9150,1.7050,2.3950,11.6500,17.9150,103.250,104.800,8.3000
2,adamsst01,2018-04-11,MEM,C,Steven Adams,OKC,76,1,home,2189,...,30.4900,20.0125,11.9350,1.5275,2.5275,0.0000,17.4225,110.050,101.675,9.1600
3,adebaba01,2018-04-11,TOR,C,Bam Adebayo,MIA,69,0,home,1097,...,23.2100,15.2050,10.6475,1.5100,3.0700,0.0000,17.8950,117.175,105.950,9.3350
4,afflaar01,2018-04-11,WAS,SG,Arron Afflalo,ORL,53,0,home,1027,...,11.9750,6.7250,18.0925,1.2950,0.4150,6.3725,21.5300,117.000,108.525,8.5900
5,aldrico01,2018-04-09,MEM,C,Cole Aldrich,MIN,21,0,home,63,...,29.0350,19.4400,12.5750,2.0200,2.6300,0.0000,17.9000,111.750,98.800,7.6200
6,aldrila01,2018-04-11,NOP,C,LaMarcus Aldridge,SAS,75,1,away,1352,...,19.2550,13.3500,10.3150,1.5250,1.9550,0.0000,16.6650,90.500,109.500,6.2900
7,allenja01,2018-04-11,BOS,C,Jarrett Allen,BRK,72,1,away,1160,...,17.1975,12.8225,4.5200,1.3300,1.4300,0.0000,14.4575,106.100,106.100,5.8750
8,allenka01,2018-04-11,BRK,PG,Kadeem Allen,BOS,18,0,home,1468,...,11.4500,6.5150,20.4850,1.3600,0.4400,0.0000,19.1550,111.400,111.250,7.9850
9,allento01,2017-12-10,PHI,SF,Tony Allen,NOP,22,0,home,76,...,14.4050,8.3100,6.7850,1.8250,0.2500,0.0000,17.4750,101.650,109.100,5.8000


In [175]:
tmp = """WITH
distinct_player_info AS(
select player, bbrefID, pos
from(
select player, bbrefID, pos, g, mp, max(g) over(partition by player, bbrefID) as max_g, max(mp) over(partition by player, bbrefID) as max_mp
from `scarlet-labs.basketball.playerinfo{season}_{partition_date}`
where tm != 'TOT'
)
where g = max_g and mp = max_mp
),

player_logs_full as(
select date(timestamp(date)) as date, bbrefID, player, pos, tm, opp, G, GS, secs_played, venue,
plus_minus,
dk
from `scarlet-labs.basketball.standard{season}_{partition_date}`
join(select * from distinct_player_info)
using(bbrefID)
),

advanced_logs_full AS(
select date(timestamp(date)) as date, bbrefID, player, pos, tm, opp, G, GS, venue,
CAST(TS_pct as float64) as TS_pct,
CAST(eFG_pct as float64) as eFG_pct,
CAST(ORB_pct as float64) as ORB_pct, CAST(DRB_pct as float64) as DRB_pct, CAST(TRB_pct as float64) as TRB_pct, CAST(AST_pct as float64) as AST_pct, CAST(STL_pct as float64) as STL_pct,
CAST(BLK_pct as float64) as BLK_pct, CAST(TOV_pct as float64) as TOV_pct, CAST(USG_pct as float64) as USG_pct,
cast(ORtg as float64) as ORtg, cast(DRtg as float64) as DRtg, cast(GmSc as float64) as GmSc
from `scarlet-labs.basketball.advanced{season}_{partition_date}`
join(select * from distinct_player_info)
using(bbrefID)
)

select *
from distinct_player_info
join(select * from player_logs_full)
using(bbrefID, player, pos)
join(select * from advanced_logs_full)
using(date, bbrefID, player, pos, tm, opp, G, GS, venue)
order by date, bbrefID"""

In [176]:
pd.to_pickle(tmp, 'get_player_data.pkl')